This notebook is responsible for visualizing the data from the Github API, which should be provided in a serialized format.

Requirement: It must be possible to run this notebook using JupyterLite.

For an exemplary use case, please see the project's `README.md`.

Important aspects of the different cells, and how to use them are explained in markdown comments above them.

### Troubleshooting:

#### Even after reloading (deep-refreshing) the page, a notebook is not updated

This likely occurred because the backend and frontend of jupyterlite are out of sync.
I am not yet sure why this happens even after a deep-refresh of the page, as this should update the frontend according to the jupyterlite documentation.
As a workaround, delete the cache/cookies for the page and reload it.
Note that you this will reset all notebooks to their versions saved on Github, so download your notebooks if necessary.
You will also have to re-run the notebooks.

An additional necessary step may be to "delete" the files in question in jupyterlite, which should refresh them with the version on Github.

#### The `commits.json` cannot be found when it is being loaded

Same reasoning & workaround as above.

In [ ]:
# We need to install dependecies for the frontend separately in JupyterLite
%pip install ipywidgets plotly pandas tabulate

In [ ]:
import json
from datetime import datetime
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets
from IPython import display
import tabulate #https://pypi.org/project/tabulate/

The `commits.json` must be provided beforehand.
We recommend using the Github CLI, using the [commits](https://docs.github.com/en/rest/commits/commits?apiVersion=2022-11-28) endpoint to generate the commit list.
You can take a look at the provided file for an example.

In [ ]:
#
# Process commit data
#
# Read commit data from 'commits.json'
with open('commits.json') as f:
    all_commits = json.load(f)
    
# Convert dates to Python datetime objects
for commit in all_commits:
    commit['commit']['author']['date'] = datetime.strptime(commit['commit']['author']['date'], '%Y-%m-%dT%H:%M:%SZ')

# Exclude merge commits, i.e. commits with more than one parent
merges = [commit for commit in all_commits if len(commit['parents']) >= 2]
commits = [commit for commit in all_commits if len(commit['parents']) <= 1]

# Exclude commits without "author" object, i.e. GitHub user was not mapped
missing = [commit for commit in commits if commit['author'] is None]
commits = [commit for commit in commits if commit['author'] is not None and len(commit['author']) > 0]

# Calculate statistics
author_logins = set([commit['author']['login'] for commit in commits])
earliest = min([commit['commit']['author']['date'] for commit in commits])
latest = max([commit['commit']['author']['date'] for commit in commits])

display.display(tabulate.tabulate(
    [["Retrieved commits from repository:",len(commits)],
     ["Merge commits that are excluded:",len(merges)],
     ["Non-merge commits missing GitHub user:",len(missing)],
     ["Amount of qualifying commits:", len(commits)],
     ["Unique authors in qualifying commits:", len(author_logins)],
     ["Earliest commit:", earliest.date()],
     ["Latest commit:", latest.date()]
    ], tablefmt='html'))
print(f"Unassigned users: {set([(c['commit']['author']['name'],c['commit']['author']['email']) for c in missing])}")

In [ ]:
#
# Input data ranges of sprints
#
sprints_out = ipywidgets.Output()
sprints_out.append_display_data(ipywidgets.Label(value="Set the number of sprints to be used as the basis of analysis:"))
num_sprints = ipywidgets.BoundedIntText(value=1, min=1, step=1, description='Number of sprints:', style=dict(description_width='initial'))
sprints_out.append_display_data(num_sprints)
sprints_out.append_display_data(ipywidgets.Label(value="Set start and end dates for each sprint:"))
sprint_names, start_dates, end_dates = [], [], []

# Initialize the widgets for the first sprint
sprint_names.append(ipywidgets.Text(value="All time", placeholder='Name to be displayed', description='Sprint Name:'))
start_dates.append(ipywidgets.DatePicker(value=earliest.date(), description='Start Date'))
end_dates.append(ipywidgets.DatePicker(value=latest.date(), description='End Date'))
sprints_out.append_display_data(sprint_names[0])
sprints_out.append_display_data(start_dates[0])
sprints_out.append_display_data(end_dates[0])

# Automatically add/remove the widgets based on the number of sprints
def update_sprints(change):
	if change['type'] == 'change' and change['name'] == 'value':
		if change['new'] > change['old']:
			# Only add a new widget if we do not have one with this "number" already
			if len(sprint_names) < change['new']:
				sprint_names.append(ipywidgets.Text(value=f'Sprint {change["new"]}', placeholder='Set the name for this Sprint', description='Sprint Name:'))
				start_dates.append(ipywidgets.DatePicker(description='Start Date'))
				end_dates.append(ipywidgets.DatePicker(description='End Date'))
			# Display the necessary widgets for the new sprint
			sprints_out.append_display_data(sprint_names[change["new"] - 1])
			sprints_out.append_display_data(start_dates[change["new"] - 1])
			sprints_out.append_display_data(end_dates[change["new"] - 1])
		elif change['new'] < change['old']:
			# Remove the widgets for the removed sprint (workaround, as clear_output() will not work here, as we are using append_display_data() instead of the 'with out:' syntax)
			sprints_out.outputs = sprints_out.outputs[:-3]

num_sprints.observe(update_sprints, names='value')

display.display(sprints_out)

In [ ]:
#
# Input teams and team members
#
teams_output = ipywidgets.Output()
teams_output.append_display_data(ipywidgets.Label(value="Set the number of teams using the slider, and the text fields to set names for each team."))
num_teams = ipywidgets.IntSlider(min=1, max=10, step=1, description='No. of teams')
teams_output.append_display_data(num_teams)
teams_output.append_display_data(ipywidgets.Label(value="Upload a JSON file of with a list of team mebers in the format `['githubUsername', 'anotherUsername']`"))

team_names = []
team_files = []
team_members = []

# Handler for the file upload widgets
def handle_upload(change):
	# Only handle the upload if the file is not empty
	if change['new']:
		# Get the team index from the name of the widget
		team_index = int(change['owner'].description.split(' ')[2]) - 1
		# Get the file name and content
		uploaded_file = next(iter(team_files[team_index].value))
		with open(uploaded_file.name, 'wb') as f:
			f.write(uploaded_file.content)
		with open(uploaded_file.name) as f:
			team_members[team_index] = json.load(f)
		print(f'Uploaded file {uploaded_file.name} for team {team_names[i].value}')
		# Print the team members
		print(f'Team members:')
		print("\n".join(team_members[team_index]))
		print()

# Initialize the widgets for the first team
team_names.append(ipywidgets.Text(value=f'Team 1', placeholder='Set the name for this Team', description='Team Name:'))
team_files.append(ipywidgets.FileUpload(description=f'Upload Team 1', accept='.json'))
team_members.append([])
teams_output.append_display_data(team_names[0])
teams_output.append_display_data(team_files[0])

# Register the upload handler for the first team
team_files[0].observe(handle_upload, names='value')

# Automatically add/remove the widgets based on the number of teams
def update_teams(change):
	if change['type'] == 'change' and change['name'] == 'value':
		if change['new'] > change['old']:
			# Only add a new widget if we do not have one with this "number" already
			if len(team_names) < change['new']:
				team_names.append(ipywidgets.Text(value=f'Team {change["new"]}', placeholder='Set the name for this Team', description='Team Name:'))
				team_files.append(ipywidgets.FileUpload(description=f'Upload Team {change["new"]}', accept='.json'))
				team_members.append([])
				# Register the upload handler for the new team
				team_files[-1].observe(handle_upload, names='value')
			# Display the necessary widgets for the new team
			teams_output.append_display_data(team_names[change["new"] - 1])
			teams_output.append_display_data(team_files[change["new"] - 1])
		elif change['new'] < change['old']:
			# Remove the widgets for the removed team (workaround, as clear_output() will not work here, as we are using append_display_data() instead of the 'with out:' syntax)
			teams_output.outputs = teams_output.outputs[:-2]

num_teams.observe(update_teams, names='value')

display.display(teams_output)

In [ ]:
out = {}
for i, team in enumerate(team_members):
    out[team_names[i].value] = []
    for login in team:
        matching_commits = [c for c in commits if c['author']['login'] == login]
        # out.append(display.Image(url=matching_commit[0]['author']['avatar_url'], width = 25))
        out[team_names[i].value].append({'login': login, 'commit_count': len(matching_commits)})
        
for team, members in out.items():
    fig = go.Figure(go.Bar(
                x=[e['commit_count'] for e in members],
                y=[e['login'] for e in members],
                orientation='h'))
    fig.update_layout(title=team, xaxis_title="Amount of commits")
    fig.show()

This cell contains the logic for the commit heatmap.
As with any other cell, you can directly edit the code to change the tool's behaviour in ways not supported by the UI.
For example, you could change the color scheme from `oranges` to `viridis` by changing the `color_continuous_scale` parameter in the `px.densitytmap` call.

In [ ]:
def get_time_of_day(date):
	# Get the hour of the day
	hour = date.hour
	# Return the time of day
	if hour <= 7 or hour >= 22:
		return 0 # "night"
	elif hour <= 12:
		return 1 # "morning"
	elif hour <= 17:
		return 2 # "afternoon"
	else:
		return 3 # "evening"

# Plot a heatmap for the commit timings
def plot_commit_heatmap(commits, team_members=None, start_date=None, end_date=None, time_mode="time_of_day"):
	# Convert start and and dates to datetime objects
	if start_date is not None:
		start_date = datetime.combine(start_date, datetime.min.time())
	if end_date is not None:
		end_date = datetime.combine(end_date, datetime.max.time())

	# Only include commits authored by one of the team members
	if team_members is not None:
		commit_dates = [commit['commit']['author']['date'] for commit in commits if (commit['author']['login'] in team_members)]
	else:
		commit_dates = [commit['commit']['author']['date'] for commit in commits]
	
	# Filter the dates to only include those between the start and end dates
	commit_dates = [date for date in commit_dates if (start_date is None or date >= start_date) and (end_date is None or date <= end_date)]

	# Get the day of each commit
	days = [date.weekday() for date in commit_dates]

	# Depending on the time_mode, get the hour or time of day for each commit
	if time_mode == 'hour':
		x_vals = [date.hour for date in commit_dates]
		x_axis_title = 'Hour'
		x_axis_tick_vals = np.arange(0, 24, 1)
		x_axis_tick_text = np.arange(0, 24, 1)
		x_bins = 24
	elif time_mode == 'time_of_day':
		x_vals = [get_time_of_day(date) for date in commit_dates]
		x_axis_title = 'Time of day'
		x_axis_tick_vals = np.arange(0, 4)
		x_axis_tick_text = ['Night (22-7)', 'Morning (7-12)', 'Afternoon (12-17)', 'Evening (17-22)']
		x_bins = 4
	
	if len(x_vals) > 0:
		# Create a plotly heatmap
		fig = px.density_heatmap(x=x_vals, y=days, nbinsx=x_bins, nbinsy=7, histfunc='count', color_continuous_scale='oranges')
	else:
		# An empty array for x would cause an error, so we create a dummy heatmap with a single point
		fig = px.density_heatmap(x=[0], y=[], nbinsx=1, nbinsy=7, histfunc='count', color_continuous_scale='oranges')

	# Set y axis labels
	fig.update_yaxes(tickvals=np.arange(0, 7), ticktext=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

	# Set the title and axis labels
	fig.update_layout(title='Commit timings', xaxis_title=x_axis_title, yaxis_title='Day of the week', xaxis_tickvals=x_axis_tick_vals, xaxis_ticktext=x_axis_tick_text)

	# Update the labels for the hovertext. It should show the number of commits, the time and day of the week
	fig.update_traces(hovertemplate='%{z} commit(s) on %{y}' + ('<br>%{x}' if time_mode == 'time_of_day' else '<br>starting at %{x} o\'clock'))

	# In order to be able to update the displayed figure within the cell's output (instead of the logs), we need to wrap it in a FigureWidget
	displayed_fig = go.FigureWidget(fig)
	return displayed_fig

This cell handles the display of the heatmap for the current project.
Only one heatmap is displayed at a time, the sprint, team and "time-mode" for which this is done can be selected using the dropdown menus.
If you change the date ranges of the sprints, you *do not* have to re-run this cell - any newly generated heatmap will automatically use the new date ranges.
However, you do need to re-run the cell if you have changed the number of sprints, as this is not currently handled automatically.
The same is true when editing the teams and team members.

In [ ]:
heatmap_output = ipywidgets.Output()

# Widget to select the displayed sprint
sprint_selector = ipywidgets.Dropdown(
	options=[(sprint_names[i].value, i) for i in range(num_sprints.value)],
	description='Sprint:'
)

# Widget to select the displayed team
team_selector = ipywidgets.Dropdown(
	options=[('All Authors', -1)] + [(team_names[i].value, i) for i in range(num_teams.value)],
	description='Team:'
)
selected_team_name = team_names[team_selector.value].value if team_selector.value != -1 else 'All Authors'
selected_team_members = team_members[team_selector.value] if team_selector.value != -1 else None

# Widget to select which time-mode to display
time_mode_selector = ipywidgets.Dropdown(
	options=[('Hourly', 'hour'), ('By time of day', 'time_of_day')],
	description='Time mode:'
)

fig = plot_commit_heatmap(commits, selected_team_members, start_dates[sprint_selector.value].value, end_dates[sprint_selector.value].value, time_mode_selector.value)
fig.update_layout(title=f"Commit times in '{sprint_names[sprint_selector.value].value}' ({start_dates[sprint_selector.value].value} - {end_dates[sprint_selector.value].value}) for '{selected_team_name}'")

heatmap_output.append_display_data(sprint_selector)
heatmap_output.append_display_data(team_selector)
heatmap_output.append_display_data(time_mode_selector)
heatmap_output.append_display_data(fig)

def update_fig(change):
	selected_team_name = team_names[team_selector.value].value if team_selector.value != -1 else 'All Authors'
	selected_team_members = team_members[team_selector.value] if team_selector.value != -1 else None
	fig = plot_commit_heatmap(commits, selected_team_members, start_dates[sprint_selector.value].value, end_dates[sprint_selector.value].value, time_mode_selector.value)
	fig.update_layout(title=f"Commit times in '{sprint_names[sprint_selector.value].value}' ({start_dates[sprint_selector.value].value} - {end_dates[sprint_selector.value].value}) for '{selected_team_name}'")
	# Remove the last item from the output, which is the old figure
	heatmap_output.outputs = heatmap_output.outputs[:-1]
	# Add the new figure to the output
	heatmap_output.append_display_data(fig)

sprint_selector.observe(update_fig, names='value')
team_selector.observe(update_fig, names='value')
time_mode_selector.observe(update_fig, names='value')

display.display(heatmap_output)